In [ ]:
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)

import matplotlib.pyplot as plt
import numpy as np

from fct import *

from collections import Counter
from scipy import misc
from sklearn import cluster

from skimage.measure import label
from skimage.morphology import remove_small_objects, dilation, disk
from skimage.exposure import equalize_adapthist
from skimage.feature import canny

# Clean iris
from skimage.morphology import (convex_hull_image, binary_erosion, 
                                dilation, erosion)
from skimage import color
# Find center
from scipy.signal import convolve2d as conv2

In [ ]:
img1 = read_img('001', 'L', '1')
img2 = read_img('009', 'L', '4')
img3 = read_img('003', 'L', '2')
img4 = read_img('004', 'L', '2')

#
#   Images
#
plt.gray()
fig, ax = plt.subplots(2, 2)
plt.gray()
ax1, ax2, ax3, ax4 = ax.ravel()
ax1.imshow(img1)
ax1.set_title('img1')
ax2.imshow(img2)
ax2.set_title('img2')
ax3.imshow(img3)
ax3.set_title('img3')
ax4.imshow(img4)
ax4.set_title('img4')


#
#   Histograms
#
fig, ax = plt.subplots(2, 2)
ax1, ax2, ax3, ax4 = ax.ravel()
ax1.hist(img1.flatten(), 256)
ax1.set_title('img1 kmeans histo')
ax2.hist(img2.flatten(), 256)
ax2.set_title('img2 kmeans histo')
ax3.hist(img3.flatten(), 256)
ax3.set_title('img3 kmeans histo')
ax4.hist(img4.flatten(), 256)
ax4.set_title('img4 kmeans histo')

## Pupil detection

### segment pupil

In [ ]:
nb_clusters = 22
img1_k = kmeans(img1, nb_clusters)
img2_k = kmeans(img2, nb_clusters)
img3_k = kmeans(img3, nb_clusters)
img4_k = kmeans(img4, nb_clusters)

#
#   Kmeans
#
fig, ax = plt.subplots(2, 2)
plt.gray()
ax1, ax2, ax3, ax4 = ax.ravel()
ax1.imshow(img1_k)
ax1.set_title('img1 kmeans')
ax2.imshow(img2_k)
ax2.set_title('img2 kmeans')
ax3.imshow(img3_k)
ax3.set_title('img3 kmeans')
ax4.imshow(img4_k)
ax4.set_title('img4 kmeans')

#
#   Histograms
#
fig, ax = plt.subplots(2, 2)
ax1, ax2, ax3, ax4 = ax.ravel()
ax1.hist(img1_k.flatten(), 256)
ax1.set_title('img1 kmeans histo')
ax2.hist(img2_k.flatten(), 256)
ax2.set_title('img2 kmeans histo')
ax3.hist(img3_k.flatten(), 256)
ax3.set_title('img3 kmeans histo')
ax4.hist(img4_k.flatten(), 256)
ax4.set_title('img4 kmeans histo')

In [ ]:
#
#   Pupil is the darkest part of the IR image
#
darkest_cluster1 = segment_lowest_cluster(img1_k)
darkest_cluster2 = segment_lowest_cluster(img2_k)
darkest_cluster3 = segment_lowest_cluster(img3_k)
darkest_cluster4 = segment_lowest_cluster(img4_k)

#
#   Calcualte mean of the darkest cluster
#
x1, y1 = cluster_mean(darkest_cluster1)
x2, y2 = cluster_mean(darkest_cluster2)
x3, y3 = cluster_mean(darkest_cluster3)
x4, y4 = cluster_mean(darkest_cluster4)

#
#   Plot Lowest cluster and mean
#
fig, ax = plt.subplots(2, 2)
plt.gray()
ax1, ax2, ax3, ax4 = ax.ravel()
ax1.imshow(darkest_cluster1)
ax1.scatter(y1, x1, color='r')
ax1.set_title('img1 darkest_cluster')
ax2.imshow(darkest_cluster2)
ax2.scatter(y2, x2, color='r')
ax2.set_title('img2 darkest_cluster')
ax3.imshow(darkest_cluster3)
ax3.scatter(y3, x3, color='r')
ax3.set_title('img3 darkest_cluster')
ax4.imshow(darkest_cluster4)
ax4.scatter(y4, x4, color='r')
ax4.set_title('img4 darkest_cluster')

In [ ]:
#
#   Get clean image of pupil
#
clean_pupil1, rad1 = find_pupil(darkest_cluster1)
clean_pupil2, rad2 = find_pupil(darkest_cluster2)
clean_pupil3, rad3 = find_pupil(darkest_cluster3)
clean_pupil4, rad4 = find_pupil(darkest_cluster4)
    
fig, ax = plt.subplots(2, 2)
plt.gray()
ax1, ax2, ax3, ax4 = ax.ravel()
ax1.imshow(clean_pupil1)
ax1.set_title('img1 clean pupil')
ax2.imshow(clean_pupil2)
ax2.set_title('img2 clean pupil')
ax3.imshow(clean_pupil3)
ax3.set_title('img3 clean pupil')
ax4.imshow(clean_pupil4)
ax4.set_title('img4 clean pupil')

### From segmented pupil get center and radius

In [ ]:
#
#   Find center
#
center1_x, center1_y = center(clean_pupil1, rad1)
center2_x, center2_y = center(clean_pupil2, rad2)
center3_x, center3_y = center(clean_pupil3, rad3)
center4_x, center4_y = center(clean_pupil4, rad4)

#
#   Adjust radius
#
rad1 = radius(clean_pupil1, center1_x, center1_y)
rad2 = radius(clean_pupil2, center2_x, center2_y)
rad3 = radius(clean_pupil3, center3_x, center3_y)
rad4 = radius(clean_pupil4, center4_x, center4_y)


#
#   Plot both
#
fig, ax = plt.subplots(1, 2)
plt.gray()
ax1, ax2 = ax.ravel()
image = color.gray2rgb(clean_pupil1)
circle = plt.Circle((center1_y, center1_x), rad1, fill=False, color='r')
ax1.add_artist(circle)
ax1.scatter(center1_y, center1_x, color='r')
ax1.imshow(image)
image = color.gray2rgb(img1)
circle = plt.Circle((center1_y, center1_x), rad1, fill=False, color='r')
ax2.add_artist(circle)
ax2.scatter(center1_y, center1_x, color='r')
ax2.imshow(image)

fig, ax = plt.subplots(1, 2)
plt.gray()
ax1, ax2 = ax.ravel()
image = color.gray2rgb(clean_pupil2)
circle = plt.Circle((center2_y, center2_x), rad2, fill=False, color='r')
ax1.add_artist(circle)
ax1.scatter(center2_y, center2_x, color='r')
ax1.imshow(image)
image = color.gray2rgb(img2)
circle = plt.Circle((center2_y, center2_x), rad2, fill=False, color='r')
ax2.add_artist(circle)
ax2.scatter(center2_y, center2_x, color='r')
ax2.imshow(image)

fig, ax = plt.subplots(1, 2)
plt.gray()
ax1, ax2 = ax.ravel()
image = color.gray2rgb(clean_pupil3)
circle = plt.Circle((center3_y, center3_x), rad3, fill=False, color='r')
ax1.add_artist(circle)
ax1.scatter(center3_y, center3_x, color='r')
ax1.imshow(image)
image = color.gray2rgb(img3)
circle = plt.Circle((center3_y, center3_x), rad3, fill=False, color='r')
ax2.add_artist(circle)
ax2.scatter(center3_y, center3_x, color='r')
ax2.imshow(image)

fig, ax = plt.subplots(1, 2)
plt.gray()
ax1, ax2 = ax.ravel()
image = color.gray2rgb(clean_pupil4)
circle = plt.Circle((center4_y, center4_x), rad4, fill=False, color='r')
ax1.add_artist(circle)
ax1.scatter(center4_y, center4_x, color='r')
ax1.imshow(image)
image = color.gray2rgb(img4)
circle = plt.Circle((center4_y, center4_x), rad4, fill=False, color='r')
ax2.add_artist(circle)
ax2.scatter(center4_y, center4_x, color='r')
ax2.imshow(image)

plt.figure()
image = color.gray2rgb(img1)
circle = plt.Circle((center1_y, center1_x), rad1, fill=False, color='r')
plt.gca().add_artist(circle)
plt.scatter(center1_y, center1_x, color='r')
plt.imshow(image)

plt.figure()
image = color.gray2rgb(img2)
circle = plt.Circle((center2_y, center2_x), rad2, fill=False, color='r')
plt.gca().add_artist(circle)
plt.scatter(center2_y, center2_x, color='r')
plt.imshow(image)

plt.figure()
image = color.gray2rgb(img3)
circle = plt.Circle((center3_y, center3_x), rad3, fill=False, color='r')
plt.gca().add_artist(circle)
plt.scatter(center3_y, center3_x, color='r')
plt.imshow(image)

plt.figure()
image = color.gray2rgb(img4)
circle = plt.Circle((center4_y, center4_x), rad4, fill=False, color='r')
plt.gca().add_artist(circle)
plt.scatter(center4_y, center4_x, color='r')
plt.imshow(image)

## Iris Detection

In [ ]:
def bin_image(img):

    it = np.copy(img)
    it_ad = equalize_adapthist(it)

    # Kmeans to label different area
    nb_clusters = 20
    itk = kmeans(it, nb_clusters)
    itk_ad = kmeans(it_ad, nb_clusters)

    # Keep only iris and pupil
    nb_clusters_to_keep = 7
    t_ad = list(set(itk_ad.flatten()))
    t_ad.sort()
    threshold = t_ad[nb_clusters_to_keep]
    ii_ad = itk_ad <= threshold
    ii_ad = dilation(erosion(ii_ad, disk(5)), disk(5))
    
    return ii_ad

In [ ]:
#
#   Segment iris and pupil
#
ii_ad1 = bin_image(img1)
ii_ad1_canny = canny(ii_ad1)
ii_ad2 = bin_image(img2)
ii_ad2_canny = canny(ii_ad2)
ii_ad3 = bin_image(img3)
ii_ad3_canny = canny(ii_ad3)
ii_ad4 = bin_image(img4)
ii_ad4_canny = canny(ii_ad4)


#
#   Plot cicles with binarized image
#
fig, ax = plt.subplots(2, 2)
plt.gray()
ax1, ax2, ax3, ax4 = ax.ravel()
ax1.imshow(color.gray2rgb(ii_ad1))
circle = plt.Circle((center1_y, center1_x), rad1, fill=False, color='r')
ax1.add_artist(circle)
circle = plt.Circle((center1_y, center1_x), 2 * rad1, fill=False, color='g')
ax1.add_artist(circle)
circle = plt.Circle((center1_y, center1_x), 330, fill=False, color='b')
ax1.add_artist(circle)
ax1.scatter(center1_y, center1_x, color='r')
ax2.imshow(color.gray2rgb(ii_ad2))
circle = plt.Circle((center2_y, center2_x), rad2, fill=False, color='r')
ax2.add_artist(circle)
circle = plt.Circle((center2_y, center2_x), 2 * rad2, fill=False, color='g')
ax2.add_artist(circle)
circle = plt.Circle((center2_y, center2_x), 330, fill=False, color='b')
ax2.add_artist(circle)
ax2.scatter(center2_y, center2_x, color='r')
ax3.imshow(color.gray2rgb(ii_ad3))
circle = plt.Circle((center3_y, center3_x), rad3, fill=False, color='r')
ax3.add_artist(circle)
circle = plt.Circle((center3_y, center3_x), 2 * rad3, fill=False, color='g')
ax3.add_artist(circle)
circle = plt.Circle((center3_y, center3_x), 330, fill=False, color='b')
ax3.add_artist(circle)
ax3.scatter(center3_y, center3_x, color='r')
ax4.imshow(color.gray2rgb(ii_ad4))
circle = plt.Circle((center4_y, center4_x), rad4, fill=False, color='r')
ax4.add_artist(circle)
circle = plt.Circle((center4_y, center4_x), 3 * rad4, fill=False, color='g')
ax4.add_artist(circle)
circle = plt.Circle((center4_y, center4_x), 330, fill=False, color='b')
ax4.add_artist(circle)
ax4.scatter(center4_y, center4_x, color='r')

In [ ]:
def radius_iris(ii_ad_canny, center_x, center_y, rad, plot_hist=False):

    distances = []
    for i in range(ii_ad_canny.shape[0]):
        for j in range(ii_ad_canny.shape[1]):
            if i > center_x and ii_ad_canny[i,j] != 0:
                dist = np.sqrt((i - center_x) ** 2 + (j - center_y) ** 2)
                # Keep only edges that are between two circles defined by
                # two times the radius and 350
                if 2 * rad < dist < 350:
                    distances.append(dist)
    
    hist = np.histogram(distances, bins=50)
    if plot_hist:
        plt.hist(distances, bins=50)
        plt.show()

    return hist[1][np.argmax(hist[0])]

In [ ]:
#
#   Get radius of the iris
#
r1 = radius_iris(ii_ad1_canny, center1_x, center1_y, rad1, plot_hist=True)
r2 = radius_iris(ii_ad2_canny, center2_x, center2_y, rad2)
r3 = radius_iris(ii_ad3_canny, center3_x, center3_y, rad3)
r4 = radius_iris(ii_ad4_canny, center4_x, center4_y, rad4)


#
#   Plot final result
#
fig, ax = plt.subplots(2, 2)
plt.gray()
ax1, ax2, ax3, ax4 = ax.ravel()
ax1.imshow(color.gray2rgb(img1))
circle = plt.Circle((center1_y, center1_x), rad1, fill=False, color='r')
ax1.add_artist(circle)
circle = plt.Circle((center1_y, center1_x), r1, fill=False, color='r')
ax1.add_artist(circle)
ax1.scatter(center1_y, center1_x, color='r')
ax2.imshow(color.gray2rgb(img2))
circle = plt.Circle((center2_y, center2_x), rad2, fill=False, color='r')
ax2.add_artist(circle)
circle = plt.Circle((center2_y, center2_x), r2, fill=False, color='r')
ax2.add_artist(circle)
ax2.scatter(center2_y, center2_x, color='r')
ax3.imshow(color.gray2rgb(img3))
circle = plt.Circle((center3_y, center3_x), rad3, fill=False, color='r')
ax3.add_artist(circle)
circle = plt.Circle((center3_y, center3_x), r3, fill=False, color='r')
ax3.add_artist(circle)
ax3.scatter(center3_y, center3_x, color='r')
ax4.imshow(color.gray2rgb(img4))
circle = plt.Circle((center4_y, center4_x), rad4, fill=False, color='r')
ax4.add_artist(circle)
circle = plt.Circle((center4_y, center4_x), r4, fill=False, color='r')
ax4.add_artist(circle)
ax4.scatter(center4_y, center4_x, color='r')